In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket_tmp/crocket')

from datetime import datetime, timedelta
from multiprocessing import Queue, Process
from numpy import mean, median
from os import environ
from os.path import join
from time import sleep, time as time_now

from crocket.sql.sql import Database
from crocket.utilities.credentials import get_credentials

In [2]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
DATABASE_NAME = 'BITTREX3'

In [3]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

# Create database connection
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [4]:
skip_list = ['BTC-BCC', 'BTC-ETH', 'BTC-LSK', 'BTC-NEO', 'BTC-XRP', 'BTC-LTC']

In [5]:
def run_algorithm(data, status,
                  duration=3, 
                  price_lag_time=30, 
                  price_lag_duration=5,
                  price_lag_threshold=0.05,
                  volume_lag_duration=30,
                  volume_lag_threshold=2,
                  profit_percent=0.05, 
                  stop_loss_percent=0.01, 
                  stop_gain_percent=0.02, 
                  max_hold_time=10800, 
                  wait_time=14400):
    
    time = data.get('time')
    buyvolume = data.get('buyvolume')
    wprice = data.get('wprice')
    
    last_buy_time_difference = (time[-1] - status.get('last_buy').get('start')).total_seconds()
    
    # Action if haven't bought coin
    if not status.get('bought'):

        # No action if purchased within time of last buy
        if last_buy_time_difference < wait_time:
            return status

        sample_volume_mean = mean(buyvolume[-duration:])
        volume_lag_median = median(buyvolume[-(duration+volume_lag_duration):-duration])

        if sample_volume_mean > 0 and volume_lag_median < volume_lag_threshold:
            sample_price = float(wprice[-1])
            previous_price = float(mean(wprice[-(duration+price_lag_time):-(duration+price_lag_time-price_lag_duration)]))

            if sample_volume_mean > 2 and \
            abs((sample_price - previous_price)/previous_price) < price_lag_threshold and \
            sum([1 if x > 1 else 0 for x in buyvolume[-duration:]]) >= 3:
                
                # TODO: MAKE API CALL TO BUY (wrap in try)
                status['bought'] = True
                status['current_buy'] = {'start': time[-1],
                                         'buy_price': float(wprice[-1])}

    # Action if have bought coin
    else:
        current_buy = status.get('current_buy')
        current_buy_hold_time = (time[-1] - current_buy.get('start')).total_seconds()
        
        # Activate stop gain signal after passing threshold percentage
        if wprice[-1] > (current_buy.get('buy_price') * (stop_gain_percent + 1)):
            status['stop_gain'] = True

        # Activate maximize gain signal after passing profit threshold
        if wprice[-1] > (current_buy.get('buy_price') * (profit_percent + 1)):
            print(wprice[-1], current_buy.get('buy_price'), (current_buy.get('buy_price') * (profit_percent + 1)))
            status['maximize_gain'] = True
            
        # Sell if hit stop loss
        # Sell after hitting profit threshold followed by drop in price of X%
        # Sell after passing max hold time
        # Sell after detecting stop gain signal and price drop below stop gain price
        if (wprice[-1] < (current_buy.get('buy_price') * (1 - stop_loss_percent))) or \
        status.get('maximize_gain') or \
        current_buy_hold_time > max_hold_time or \
        (status.get('stop_gain') and wprice[-1] < (current_buy.get('buy_price') * (stop_gain_percent + 1))):
            
            # TODO: MAKE API CALL TO SELL (wrap in try)
            status['current_buy']['stop'] = time[-1]
            
            buy_price = current_buy.get('buy_price')
            sell_price = float(wprice[-1])
            
            status['current_buy']['sell_price'] = sell_price
            status['current_buy']['profit'] = ((sell_price - buy_price)/buy_price) * 0.9975**2

            status['bought'] = False
            status['stop_gain'] = False
            status['maximize_gain'] = False
            
            status['last_buy'] = {'start': current_buy.get('start'),
                                  'buy_price': buy_price}
        
    return status

In [6]:
def main_loop(iqueue, data_all):
    
    # Initialize price and status objects
        
    markets = [x for x in db.get_all_tables() if x not in skip_list]
    
    while len(data_all.get('BTC-OK').get('time')) > 0:
        
        data = {}
        
        for market in markets:
            
            if data_all.get(market):
                
                data[market] = {'time': data_all.get(market).get('time')[0],
                                'wprice': data_all.get(market).get('wprice')[0],
                                'buyvolume': data_all.get(market).get('buyvolume')[0]}
                
                del data_all[market]['time'][0]
                del data_all[market]['wprice'][0]
                del data_all[market]['buyvolume'][0]
        
        iqueue.put(data)
    
    print('finished all data')
    iqueue.put('QUIT')
        

def run_tradebot(market_queue, market_status):
    
    data = {}
    results = {}
    
    markets = [x for x in db.get_all_tables() if x not in skip_list]
    
    for market in markets:
        data[market] = {'time': [],
                        'wprice': [],
                        'buyvolume': []}
        
        results[market] = []
    
    while True:
        #try:
        market_data = market_queue.get()
        
        if market_data == 'QUIT':
            print('QUIT SIGNAL DETECTED, quiting ...')
            break
        
        for market in market_data:
            
            if market_data.get(market).get('wprice') > 0:  # Temporary fix for entries with 0 price
                data[market]['time'].append(market_data.get(market).get('time'))
                data[market]['wprice'].append(market_data.get(market).get('wprice'))
                data[market]['buyvolume'].append(market_data.get(market).get('buyvolume'))
            
        start = time_now()
        for market in market_data:
            
            if len(data.get(market).get('time')) > 60:
                
                del data[market]['time'][0]
                del data[market]['wprice'][0]
                del data[market]['buyvolume'][0]
                
                market_status[market] = run_algorithm(data.get(market), market_status.get(market))

                if market_status.get(market).get('current_buy').get('profit'):
                    results[market].append(market_status.get(market).get('current_buy').get('profit'))
                    print(market_status.get(market).get('current_buy'))
                    market_status[market]['current_buy'] = {}
                    # TODO: insert completed buy into database

        stop = time_now()
        #print('{} - Elapsed time: {}'.format(str(data.get('BTC-ADA').get('time')[0]), str(stop-start)))
            
        #except:
        #    print('Stopping tradebot ...')
        #    # TODO: exit all trades before exiting
        #    break
    
    print('Bot exited loop.')
    
    total_profit = 0
    for market in sorted(results.keys()):
        
        profit = sum(results.get(market))
        
        if profit > 0:
            total_profit += profit
            print('{}: {}'.format(market, str(profit)))

    print('Total profit: {}'.format(str(total_profit)))

In [7]:
# Get data from sql table

all_data, status = {}, {}

bought_time = datetime(2017, 11, 16, 21, 59, 3)
    
last_buy = {'start': bought_time,
            'buy_price': 0}

tables = db.get_all_tables()

for table in tables:
    
    status[table] = {'bought': False,
                     'last_buy': last_buy,
                     'current_buy': {},
                     'stop_gain': False,
                     'maximize_gain': False}

    result = db.select_query(table, '*', 'where time < "2017/11/20 20:14:00"')
    
    if result:
        time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*result)

        all_data[table] = {'time': list(time),
                           'wprice': list(wprice),
                           'buyvolume': list(buyvolume)}

In [8]:
queue = Queue()

tradebot = Process(target=run_tradebot, args=(queue,status))
tradebot.start()

main_loop(queue, all_data)

0.06315840 0.05999666 0.062996493
0.06315840 0.0593966934
1260.0 10800
SOLD. True True
{'start': datetime.datetime(2017, 11, 17, 22, 34), 'buy_price': 0.05999666, 'stop': datetime.datetime(2017, 11, 17, 22, 55), 'sell_price': 0.0631584, 'profit': 0.05243543658722008}
0.00104158 0.00099123 0.0010407915
0.00104158 0.0009813177
4920.0 10800
SOLD. True True
{'start': datetime.datetime(2017, 11, 17, 22, 52), 'buy_price': 0.00099123, 'stop': datetime.datetime(2017, 11, 18, 0, 14), 'sell_price': 0.00104158, 'profit': 0.05054181641748132}
finished all data
0.00040966 0.0003977919
{'start': datetime.datetime(2017, 11, 18, 1, 15), 'buy_price': 0.00040181, 'stop': datetime.datetime(2017, 11, 18, 1, 38), 'sell_price': 0.00040966, 'profit': 0.019439036018267346}
1380.0 10800
SOLD. False True
0.05714533 0.057321
4200.0 10800
SOLD. False False
{'start': datetime.datetime(2017, 11, 18, 4, 25), 'buy_price': 0.0579, 'stop': datetime.datetime(2017, 11, 18, 5, 35), 'sell_price': 0.05714533, 'profit': -0.0

0.00030331 0.00030676140000000004
960.0 10800
SOLD. False False
{'start': datetime.datetime(2017, 11, 19, 21, 57), 'buy_price': 0.00030986, 'stop': datetime.datetime(2017, 11, 19, 22, 13), 'sell_price': 0.00030331, 'profit': -0.021033017935519344}
0.00031492 0.00029985 0.0003148425
0.00031492 0.0002968515
2040.0 10800
SOLD. True True
{'start': datetime.datetime(2017, 11, 19, 22, 31), 'buy_price': 0.00029985, 'stop': datetime.datetime(2017, 11, 19, 23, 5), 'sell_price': 0.00031492, 'profit': 0.05000748436718361}
0.00001630 1.63746e-05
2400.0 10800
SOLD. False False
{'start': datetime.datetime(2017, 11, 19, 23, 41), 'buy_price': 1.654e-05, 'stop': datetime.datetime(2017, 11, 20, 0, 21), 'sell_price': 1.63e-05, 'profit': -0.014437817412333801}
0.00007197 7.22898e-05
60.0 10800
SOLD. False False
{'start': datetime.datetime(2017, 11, 20, 1, 26), 'buy_price': 7.302e-05, 'stop': datetime.datetime(2017, 11, 20, 1, 27), 'sell_price': 7.197e-05, 'profit': -0.014307813783894804}
0.00004543 4.4619